# Initial Creation of Red, Green, Blue files (no aggregation / merging)

In [ ]:
import xml.etree.ElementTree as ET
import datetime
import csv
import pandas as pd
import numpy as np

In [2]:
def captureRoot(filePath):
    startTime = datetime.datetime.now()
    print(str(startTime)+"| Starting Root capture for %s" % (filePath))
    root = ET.parse(filePath).getroot()
    endTime = datetime.datetime.now()
    print(str(endTime)+"| Capture Complete! Duration: %i sec" % ((endTime-startTime).seconds))
    return root

In [6]:
#Blue (Stand: HKCategoryTypeIdentifierAppleStandHour
#Green (Exercise): HKQuantityTypeIdentifierAppleExerciseTime
#Red (Active Energy): HKQuantityTypeIdentifierActiveEnergyBurned

def captureSingleRecordType(rootList, typeToCapture):
    startTime = datetime.datetime.now()
    print(str(startTime)+"| Starting Record capture for %s" % (typeToCapture))
    rtnList = [[] for _ in rootList]
    for idx, root in enumerate(rootList):
        for child in root:
            if "type" not in child.attrib.keys():
                continue
            if child.attrib['type'] == typeToCapture:
                rtnList[idx].append(child.attrib)
    endTime = datetime.datetime.now()
    print(str(endTime)+"| Capture Complete! Duration: %i sec" % ((endTime-startTime).seconds))
    return rtnList

In [7]:
alexRoot = captureRoot("alexAppleHealthData/export.xml")
jamesRoot = captureRoot("jamesAppleHealthData/export.xml")

2020-01-03 16:43:20.464066| Starting capture for alexAppleHealthData/export.xml
2020-01-03 16:48:33.475213| Capture Complete! Duration: 313 sec
2020-01-03 16:48:33.653719| Starting capture for jamesAppleHealthData/export.xml
2020-01-03 16:48:57.026247| Capture Complete! Duration: 23 sec


In [8]:
standData = captureSingleRecordType([alexRoot, jamesRoot], 'HKCategoryTypeIdentifierAppleStandHour')
exerciseData = captureSingleRecordType([alexRoot, jamesRoot], 'HKQuantityTypeIdentifierAppleExerciseTime')
activeEnergyData = captureSingleRecordType([alexRoot, jamesRoot], 'HKQuantityTypeIdentifierActiveEnergyBurned')

2020-01-03 16:48:58.103886| Starting Record capture for HKCategoryTypeIdentifierAppleStandHour
2020-01-03 16:49:48.172841| Capture Complete! Duration: 50 sec
2020-01-03 16:49:48.183287| Starting Record capture for HKQuantityTypeIdentifierAppleExerciseTime
2020-01-03 16:50:09.698521| Capture Complete! Duration: 21 sec
2020-01-03 16:50:09.705400| Starting Record capture for HKQuantityTypeIdentifierActiveEnergyBurned
2020-01-03 16:50:34.876118| Capture Complete! Duration: 25 sec


In [10]:
startTime = datetime.datetime.now()
print(str(startTime)+"| Writing Stand Data...")
pd.DataFrame(standData[0])\
    .drop(labels=['device', 'sourceName', 'sourceVersion', 'type', 'creationDate', 'startDate'], axis=1)\
    .to_csv("alexAppleHealthData/stand.csv")
pd.DataFrame(standData[1])\
    .drop(labels=['device', 'sourceName', 'sourceVersion', 'type', 'creationDate', 'startDate'], axis=1)\
    .to_csv("jamesAppleHealthData/stand.csv")
endTime = datetime.datetime.now()
print(str(endTime)+"| Stand Data Write Complete! Duration: %i sec" % ((endTime-startTime).seconds))
    
startTime = datetime.datetime.now()
print(str(startTime)+"| Writing Exercise Data...")
pd.DataFrame(exerciseData[0])\
    .drop(labels=['device', 'sourceName', 'sourceVersion', 'type', 'creationDate', 'startDate'], axis=1)\
    .to_csv("alexAppleHealthData/exercise.csv")
pd.DataFrame(exerciseData[1])\
    .drop(labels=['device', 'sourceName', 'sourceVersion', 'type', 'creationDate', 'startDate'], axis=1)\
    .to_csv("jamesAppleHealthData/exercise.csv")
endTime = datetime.datetime.now()
print(str(endTime)+"| Exercise Data Write Complete! Duration: %i sec" % ((endTime-startTime).seconds))

startTime = datetime.datetime.now()
print(str(startTime)+"| Writing Active Energy Data...")
pd.DataFrame(activeEnergyData[0])\
    .drop(labels=['device', 'sourceName', 'sourceVersion', 'type', 'creationDate', 'startDate'], axis=1)\
    .to_csv("alexAppleHealthData/activeEnergy.csv")
pd.DataFrame(activeEnergyData[1])\
    .drop(labels=['device', 'sourceName', 'sourceVersion', 'type', 'creationDate', 'startDate'], axis=1)\
    .to_csv("jamesAppleHealthData/activeEnergy.csv")
endTime = datetime.datetime.now()
print(str(endTime)+"| Active Energy Data Write Complete! Duration: %i sec" % ((endTime-startTime).seconds))

2020-01-03 16:52:12.630697| Writing Stand Data...
2020-01-03 16:52:13.279185| Stand Data Write Complete! Duration: 0 sec
2020-01-03 16:52:13.279538| Writing Exercise Data...
2020-01-03 16:52:14.865702| Exercise Data Write Complete! Duration: 1 sec
2020-01-03 16:52:14.866104| Writing Active Energy Data...
2020-01-03 16:52:44.618916| Active Energy Data Write Complete! Duration: 29 sec


# Aggregation and Merging of Red, Green, Blue files

In [11]:
import pandas as pd
import numpy as np

### Blue file (Stand)

In [66]:
alexDf = pd.read_csv('alexAppleHealthData/stand.csv', index_col=0, dtype=str)
jamesDf = pd.read_csv('jamesAppleHealthData/stand.csv', index_col=0, dtype=str)

dfList = [alexDf, jamesDf]
for i in range(len(dfList)):
    dfList[i]['endDate'] = dfList[i]['endDate'].str.slice(0,10)
    dfList[i]['value'] = dfList[i]['value'].map({'HKCategoryValueAppleStandHourStood': 1, 'HKCategoryValueAppleStandHourIdle': 0})
    dfList[i] = dfList[i].groupby("endDate").agg({'value': 'sum'}).reset_index()

In [73]:
#Unclear on why suffixes are not reversed when left=dfList[0]='Alex'
mergeDf = pd.merge(left=dfList[0], right=dfList[1], how='outer', on='endDate', suffixes={"_james", "_alex"})

In [82]:
mergeDf.to_csv("output/stand.csv", index=False)

### Green File (Exercise)

In [86]:
alexDf = pd.read_csv('alexAppleHealthData/exercise.csv', index_col=0)
jamesDf = pd.read_csv('jamesAppleHealthData/exercise.csv', index_col=0)

dfList = [alexDf, jamesDf]
for i in range(len(dfList)):
    dfList[i]['endDate'] = dfList[i]['endDate'].str.slice(0,10)
    dfList[i] = dfList[i].groupby("endDate").agg({'value': 'sum'}).reset_index()

In [89]:
#Unclear on why suffixes are not reversed when left=dfList[0]='Alex'
mergeDf = pd.merge(left=dfList[0], right=dfList[1], how='outer', on='endDate', suffixes={"_james", "_alex"})

In [91]:
mergeDf.to_csv("output/exercise.csv", index=False)

### Red File (Active Energy)

In [92]:
alexDf = pd.read_csv('alexAppleHealthData/activeEnergy.csv', index_col=0)
jamesDf = pd.read_csv('jamesAppleHealthData/activeEnergy.csv', index_col=0)

dfList = [alexDf, jamesDf]
for i in range(len(dfList)):
    dfList[i]['endDate'] = dfList[i]['endDate'].str.slice(0,10)
    dfList[i] = dfList[i].groupby("endDate").agg({'value': 'sum'}).reset_index()

/anaconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [94]:
#Unclear on why suffixes are not reversed when left=dfList[0]='Alex'
mergeDf = pd.merge(left=dfList[0], right=dfList[1], how='outer', on='endDate', suffixes={"_james", "_alex"})

In [96]:
mergeDf.to_csv("output/activeEnergy.csv", index=False)

# Aggregating the 3 files together

In [97]:
import pandas as pd
import numpy as np

In [115]:
standDf = pd.read_csv("output/stand.csv")
exerciseDf = pd.read_csv("output/exercise.csv")
activeEnergyDf = pd.read_csv("output/activeEnergy.csv")

standDf.columns = ['endDate', 'stand_alex', 'stand_james']
exerciseDf.columns = ['endDate', 'exercise_alex', 'exercise_james']
activeEnergyDf.columns = ['endDate', 'activeEnergy_alex', 'activeEnergy_james']

In [117]:
standDf.merge(exerciseDf, how='outer', on='endDate')

,endDate,stand_alex,stand_james,exercise_alex,exercise_james
0,2015-10-12,2.0,NaN,6.0,NaN
1,2015-10-13,16.0,NaN,35.0,NaN
2,2015-10-14,14.0,NaN,49.0,NaN
3,2015-10-15,14.0,NaN,29.0,NaN
4,2015-10-16,15.0,NaN,95.0,NaN
5,2015-10-17,17.0,NaN,109.0,NaN
6,2015-10-18,15.0,NaN,42.0,NaN
7,2015-10-19,14.0,NaN,31.0,NaN
8,2015-10-20,13.0,NaN,32.0,NaN
9,2015-10-21,16.0,NaN,61.0,NaN
